In [345]:
import pandas as pd
from datetime import datetime
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance

In [346]:
file_path = r"C:\Users\leere\PycharmProjects\Football_ML3\Goals\cgmbetdatabase_top_5_2020+.csv"

encodings = ['utf-8', 'latin1', 'iso-8859-1', 'cp1252']

for encoding in encodings:
    try:
        data = pd.read_csv(file_path, encoding=encoding, low_memory=False)
        print(f"Successfully read the files with encoding: {encoding}")
        break
    except UnicodeDecodeError:
        print(f"Failed to decode with encoding: {encoding}")
data

Successfully read the files with encoding: utf-8


,country,sezonul,datameci,orameci,etapa,txtechipa1,txtechipa2,scor1,scor2,scorp1,...,ballpa,ballpa1,ballpa2,gsh,gch,gsa,gca,stare,codechipa1,codechipa2
0,Fra1,21,21/08/2020,1800,1,Bordeaux,Nantes,0,0,0,...,57,52,62,0,0,0,0,J,4012,4014
1,Fra1,21,22/08/2020,2000,1,Lille,Rennes,1,1,1,...,49,40,58,0,0,0,0,J,4009,4015
2,Fra1,21,22/08/2020,1600,1,Dijon,Angers,0,1,0,...,46,51,41,0,0,0,0,J,4034,4047
3,Fra1,21,23/08/2020,1200,1,AS Monaco,Reims,2,2,1,...,24,20,28,0,0,0,0,J,4004,4039
4,Fra1,21,23/08/2020,1400,1,Lorient,Strasbourg,3,1,0,...,50,47,53,0,0,0,0,J,4025,4006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8977,Spa1,25,25/05/2025,1700,38,Girona,Atletico,0,0,0,...,0,0,0,29,29,35,14,S,3053,3003
8978,Spa1,25,25/05/2025,1700,38,Leganes,Valladolid,0,0,0,...,0,0,0,19,29,14,42,S,3067,3028
8979,Spa1,25,25/05/2025,1700,38,Vallecano,Mallorca,0,0,0,...,0,0,0,25,24,19,26,S,3034,3009
8980,Spa1,25,25/05/2025,1700,38,Real Madrid,Real Sociedad,0,0,0,...,0,0,0,50,20,17,17,S,3010,3015


In [347]:
column_dict = {
    "country": "country",
    "league": "league",
    "sezonul": "season",
    "datameci": "date",
    "orameci": "ko_time",
    "etapa": "round",
    "txtechipa1": "home_team",
    "txtechipa2": "away_team",
    "scor1": "home_goals_ft",
    "scor2": "away_goals_ft",
    "scorp1": "home_goals_ht",
    "scorp2": "away_goals_ht",
    "place1": "home_team_place_total",
    "place1a": "home_team_place_home",
    "place2": "away_team_place_total",
    "place2d": "away_team_place_away",
    "cotaa": "home_odds",
    "cotae": "draw_odds",
    "cotad": "away_odds",
    # "cotao0": "",
    # "cotao1": "",
    "cotao": "over_25_odds",
    # "cotao3": "",
    # "cotao4": "",
    # "cotau0": "",
    # "cotau1": "",
    "cotau": "under_25_odds",
    # "cotau3": "",
    # "cotau4": "",
    # "gg": "",
    # "ng": "",
    "elohomeo": "elo_home",
    "eloawayo": "elo_away",
    "formah": "form_home",
    "formaa": "form_away",
    "suth": "shots_home",
    "suth1": "shots_home_1h",
    "suth2": "shots_home_2h",
    "suta": "shots_away",
    "suta1": "shots_away_1h",
    "suta2": "shots_away_2h",
    "sutht": "shots_on_target_home",
    "sutht1": "shots_on_target_home_1h",
    "sutht2": "shots_on_target_home_2h",
    "sutat": "shots_on_target_away",
    "sutat1": "shots_on_target_away_1h",
    "sutat2": "shots_on_target_away_2h",
    "corh": "corners_home",
    "corh1": "corners_home_1h",
    "corh2": "corners_home_2h",
    "cora": "corners_away",
    "cora1": "corners_away_1h",
    "cora2": "corners_away_2h",
    "foulsh": "fouls_home",
    "foulsh1": "fouls_home_1h",
    "foulsh2": "fouls_home_2h",
    "foulsa": "fouls_away",
    "foulsa1": "fouls_away_1h",
    "foulsa2": "fouls_away_2h",
    "yellowh": "yellow_cards_home",
    "yellowh1": "yellow_cards_home_1h",
    "yellowh2": "yellow_cards_home_2h",
    "yellowa": "yellow_cards_away",
    "yellowa1": "yellow_cards_away_1h",
    "yellowa2": "yellow_cards_away_2h",
    "ballph": "possession_home",
    "ballph1": "possession_home_1h",
    "ballph2": "possession_home_2h",
    "ballpa": "possession_away",
    "ballpa1": "possession_away_1h",
    "ballpa2": "possession_away_2h",
    "gsh": "goals_scored_total_home",
    "gch": "goals_conceded_total_home",
    "gsa": "goals_scored_total_away",
    "gca": "goals_conceded_total_away",
    # "stare": "",
    # "codechipa1": "",
    # "codechipa2": ""
}

data = data.rename(columns=column_dict).filter(items=column_dict.values())
data


,country,season,date,ko_time,round,home_team,away_team,home_goals_ft,away_goals_ft,home_goals_ht,...,possession_home,possession_home_1h,possession_home_2h,possession_away,possession_away_1h,possession_away_2h,goals_scored_total_home,goals_conceded_total_home,goals_scored_total_away,goals_conceded_total_away
0,Fra1,21,21/08/2020,1800,1,Bordeaux,Nantes,0,0,0,...,43,48,38,57,52,62,0,0,0,0
1,Fra1,21,22/08/2020,2000,1,Lille,Rennes,1,1,1,...,51,60,42,49,40,58,0,0,0,0
2,Fra1,21,22/08/2020,1600,1,Dijon,Angers,0,1,0,...,54,49,59,46,51,41,0,0,0,0
3,Fra1,21,23/08/2020,1200,1,AS Monaco,Reims,2,2,1,...,76,80,72,24,20,28,0,0,0,0
4,Fra1,21,23/08/2020,1400,1,Lorient,Strasbourg,3,1,0,...,50,53,47,50,47,53,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8977,Spa1,25,25/05/2025,1700,38,Girona,Atletico,0,0,0,...,0,0,0,0,0,0,29,29,35,14
8978,Spa1,25,25/05/2025,1700,38,Leganes,Valladolid,0,0,0,...,0,0,0,0,0,0,19,29,14,42
8979,Spa1,25,25/05/2025,1700,38,Vallecano,Mallorca,0,0,0,...,0,0,0,0,0,0,25,24,19,26
8980,Spa1,25,25/05/2025,1700,38,Real Madrid,Real Sociedad,0,0,0,...,0,0,0,0,0,0,50,20,17,17


In [348]:
# Convert 'date' column to datetime object
data['date'] = pd.to_datetime(data['date'], format="%d/%m/%Y", errors='coerce')

# Order by date
data = data.sort_values(by='date')

# Filter out future dates (ensure data does not go beyond today)
today = datetime.today().date()
data = data[data['date'].dt.date <= today]

In [349]:
# Assign points based on match results
data["points_home"] = data.apply(lambda row: 3 if row["home_goals_ft"] > row["away_goals_ft"]
else (1 if row["home_goals_ft"] == row["away_goals_ft"] else 0), axis=1)

data["points_away"] = data.apply(lambda row: 3 if row["away_goals_ft"] > row["home_goals_ft"]
else (1 if row["away_goals_ft"] == row["home_goals_ft"] else 0), axis=1)
#data

In [350]:
# -----------------------------
# 1. Data Preparation
# -----------------------------

# Create a home team DataFrame
home_df = data[['country', 'season', 'date', 'home_team', 'away_team',
                'home_goals_ft', 'away_goals_ft', 'home_goals_ht', 'away_goals_ht']].copy()
home_df.rename(columns={
    'home_team': 'Team',
    'away_team': 'Opponent',
    'home_goals_ft': 'GoalsScored',
    'away_goals_ft': 'GoalsConceded',
    'home_goals_ht': 'FirstHalfGoalsScored',
    'away_goals_ht': 'FirstHalfGoalsConceded'
}, inplace=True)
home_df['is_home'] = 1

# Create an away team DataFrame
away_df = data[['country', 'season', 'date', 'away_team', 'home_team',
                'away_goals_ft', 'home_goals_ft', 'away_goals_ht', 'home_goals_ht']].copy()
away_df.rename(columns={
    'away_team': 'Team',
    'home_team': 'Opponent',
    'away_goals_ft': 'GoalsScored',
    'home_goals_ft': 'GoalsConceded',
    'away_goals_ht': 'FirstHalfGoalsScored',
    'home_goals_ht': 'FirstHalfGoalsConceded'
}, inplace=True)
away_df['is_home'] = 0

# Combine the home and away DataFrames into one team-level DataFrame
team_df = pd.concat([home_df, away_df], ignore_index=True)
# Sort by country, season, team, and date so that rolling calculations are in chronological order
team_df.sort_values(by=['country', 'season', 'Team', 'date'], inplace=True)

# Define rolling window sizes (adjust as needed)
window_long = 5  # e.g. last 5 matches for long-term trends
window_short = 3 # e.g. last 3 matches for short-term momentum

# -----------------------------
# 2. Rolling Feature Computation Functions
# -----------------------------

def compute_rolling(df_sub, prefix):
    """
    Compute rolling features on a given (sorted) DataFrame subset.
    Computes metrics for both full-time and first-half goals.
    The prefix will be used to name the computed columns.
    """
    df = df_sub.copy()

    # ----- Full-Time Goals Rolling Features -----
    # Rolling averages and standard deviations over the long window
    df[prefix + '_RollingGoalsScored_Mean'] = df['GoalsScored'].rolling(window=window_long, min_periods=1).mean().shift(1)
    df[prefix + '_RollingGoalsConceded_Mean'] = df['GoalsConceded'].rolling(window=window_long, min_periods=1).mean().shift(1)
    df[prefix + '_RollingGoalsScored_Std'] = df['GoalsScored'].rolling(window=window_long, min_periods=1).std().shift(1)
    df[prefix + '_RollingGoalsConceded_Std'] = df['GoalsConceded'].rolling(window=window_long, min_periods=1).std().shift(1)

    # Short-term rolling average for momentum
    df[prefix + '_RollingGoalsScored_Mean_Short'] = df['GoalsScored'].rolling(window=window_short, min_periods=1).mean().shift(1)
    df[prefix + '_Momentum_GoalsScored'] = df[prefix + '_RollingGoalsScored_Mean_Short'] - df[prefix + '_RollingGoalsScored_Mean']

    # ----- First-Half Goals Rolling Features -----
    df[prefix + '_RollingFirstHalfGoalsScored_Mean'] = df['FirstHalfGoalsScored'].rolling(window=window_long, min_periods=1).mean().shift(1)
    df[prefix + '_RollingFirstHalfGoalsConceded_Mean'] = df['FirstHalfGoalsConceded'].rolling(window=window_long, min_periods=1).mean().shift(1)
    df[prefix + '_RollingFirstHalfGoalsScored_Std'] = df['FirstHalfGoalsScored'].rolling(window=window_long, min_periods=1).std().shift(1)
    df[prefix + '_RollingFirstHalfGoalsConceded_Std'] = df['FirstHalfGoalsConceded'].rolling(window=window_long, min_periods=1).std().shift(1)

    df[prefix + '_RollingFirstHalfGoalsScored_Mean_Short'] = df['FirstHalfGoalsScored'].rolling(window=window_short, min_periods=1).mean().shift(1)
    df[prefix + '_Momentum_FirstHalfGoalsScored'] = df[prefix + '_RollingFirstHalfGoalsScored_Mean_Short'] - df[prefix + '_RollingFirstHalfGoalsScored_Mean']

    # Function to compute trend slope using a simple linear regression
    def compute_slope(x):
        if len(x) < 2:
            return np.nan
        xs = np.arange(len(x))
        return np.polyfit(xs, x, 1)[0]

    # Trend slopes for full-time and first-half goals
    df[prefix + '_Trend_Slope_GoalsScored'] = df['GoalsScored'].rolling(window=window_long, min_periods=2).apply(compute_slope, raw=True).shift(1)
    df[prefix + '_Trend_Slope_FirstHalfGoalsScored'] = df['FirstHalfGoalsScored'].rolling(window=window_long, min_periods=2).apply(compute_slope, raw=True).shift(1)

    # List of all computed columns to return
    computed_cols = [
        # Full-Time Goals Metrics
        prefix + '_RollingGoalsScored_Mean',
        prefix + '_RollingGoalsConceded_Mean',
        prefix + '_RollingGoalsScored_Std',
        prefix + '_RollingGoalsConceded_Std',
        prefix + '_RollingGoalsScored_Mean_Short',
        prefix + '_Momentum_GoalsScored',
        prefix + '_Trend_Slope_GoalsScored',
        # First-Half Goals Metrics
        prefix + '_RollingFirstHalfGoalsScored_Mean',
        prefix + '_RollingFirstHalfGoalsConceded_Mean',
        prefix + '_RollingFirstHalfGoalsScored_Std',
        prefix + '_RollingFirstHalfGoalsConceded_Std',
        prefix + '_RollingFirstHalfGoalsScored_Mean_Short',
        prefix + '_Momentum_FirstHalfGoalsScored',
        prefix + '_Trend_Slope_FirstHalfGoalsScored'
    ]
    return df[computed_cols]

def add_rolling_features_split(group):
    """
    For each team (grouped by country, season, and team), compute:
      - Overall rolling features (all matches)
      - Home-only rolling features (for matches where is_home == 1)
      - Away-only rolling features (for matches where is_home == 0)
    The computed features are merged back into the group DataFrame.
    """
    group = group.sort_values(by='date').reset_index(drop=True)

    # Overall metrics using all matches
    overall_features = compute_rolling(group, 'Overall')
    group = pd.concat([group, overall_features], axis=1)

    # Compute home-specific metrics (if any home matches exist)
    home_mask = group['is_home'] == 1
    if home_mask.sum() > 0:
        group_home = group.loc[home_mask].copy()
        home_features = compute_rolling(group_home, 'Home')
        for col in home_features.columns:
            group.loc[home_mask, col] = home_features[col].values

    # Compute away-specific metrics (if any away matches exist)
    away_mask = group['is_home'] == 0
    if away_mask.sum() > 0:
        group_away = group.loc[away_mask].copy()
        away_features = compute_rolling(group_away, 'Away')
        for col in away_features.columns:
            group.loc[away_mask, col] = away_features[col].values

        # ----- Additional Goal Threshold Percentages -----
    # Compute cumulative (season) and rolling (last 5 games) percentages for goals scored over thresholds.
 #----- Additional Goal Threshold Percentages -----
    # For each threshold, compute season cumulative and rolling percentages.
    for threshold in [1.5, 2.5, 3.5]:
        # ----- Overall (Team as a Whole) -----
        overall_season_col = f'Overall_Percent_Over_{threshold}'
        overall_rolling_col = f'Overall_Rolling5_Percent_Over_{threshold}'
        indicator_overall = group['GoalsScored'].gt(threshold)
        # Using shift(1) to exclude the current match:
        group[overall_season_col] = indicator_overall.shift(1).expanding(min_periods=1).mean()
        group[overall_rolling_col] = indicator_overall.shift(1).rolling(window=5, min_periods=1).mean()

        # ----- Home Matches Only -----
        season_col_home = f'Home_Percent_Over_{threshold}'
        rolling_col_home = f'Home_Rolling5_Percent_Over_{threshold}'
        if home_mask.sum() > 0:
            indicator_home = group.loc[home_mask, 'GoalsScored'].gt(threshold)
            # Compute on the home subset and assign back to the group
            group.loc[home_mask, season_col_home] = indicator_home.shift(1).expanding(min_periods=1).mean()
            group.loc[home_mask, rolling_col_home] = indicator_home.shift(1).rolling(window=5, min_periods=1).mean()

        # ----- Away Matches Only -----
        season_col_away = f'Away_Percent_Over_{threshold}'
        rolling_col_away = f'Away_Rolling5_Percent_Over_{threshold}'
        if away_mask.sum() > 0:
            indicator_away = group.loc[away_mask, 'GoalsScored'].gt(threshold)
            group.loc[away_mask, season_col_away] = indicator_away.shift(1).expanding(min_periods=1).mean()
            group.loc[away_mask, rolling_col_away] = indicator_away.shift(1).rolling(window=5, min_periods=1).mean()

            # ----- Team-specific Match Outcome Percentages -----
    # These features capture the percentage of matches in which the team has scored over a given goal threshold.
    for threshold in [0.5, 1.5, 2.5, 3.5]:
        # Overall (all matches)
        overall_col = f'TeamPct_Over_{threshold}'
        indicator_overall = group['GoalsScored'].gt(threshold)
        # Use shift(1) to exclude the current match from its own calculation.
        group[overall_col] = indicator_overall.shift(1).expanding(min_periods=1).mean()

        # Home matches only
        home_col = f'Home_TeamPct_Over_{threshold}'
        if home_mask.sum() > 0:
            indicator_home = group.loc[home_mask, 'GoalsScored'].gt(threshold)
            group.loc[home_mask, home_col] = indicator_home.shift(1).expanding(min_periods=1).mean()

        # Away matches only
        away_col = f'Away_TeamPct_Over_{threshold}'
        if away_mask.sum() > 0:
            indicator_away = group.loc[away_mask, 'GoalsScored'].gt(threshold)
            group.loc[away_mask, away_col] = indicator_away.shift(1).expanding(min_periods=1).mean()




    return group

# -----------------------------
# 3. Apply the Feature Engineering Group-Wise
# -----------------------------
# Group by country, season, and team, and then apply the feature engineering function.
team_df = team_df.groupby(['country', 'season', 'Team'], group_keys=False).apply(add_rolling_features_split).reset_index(drop=True)

# Display the first few rows of the resulting DataFrame
team_df


C:\Users\leere\AppData\Local\Temp\ipykernel_23128\4045428432.py:194: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  team_df = team_df.groupby(['country', 'season', 'Team'], group_keys=False).apply(add_rolling_features_split).reset_index(drop=True)


,country,season,date,Team,Opponent,GoalsScored,GoalsConceded,FirstHalfGoalsScored,FirstHalfGoalsConceded,is_home,...,Away_TeamPct_Over_0.5,TeamPct_Over_1.5,Home_TeamPct_Over_1.5,Away_TeamPct_Over_1.5,TeamPct_Over_2.5,Home_TeamPct_Over_2.5,Away_TeamPct_Over_2.5,TeamPct_Over_3.5,Home_TeamPct_Over_3.5,Away_TeamPct_Over_3.5
0,Eng1,21,2020-09-12,Arsenal,Fulham,3,0,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Eng1,21,2020-09-19,Arsenal,West Ham,2,1,1,1,1,...,NaN,1.000000,NaN,NaN,1.000000,NaN,NaN,0.000000,NaN,NaN
2,Eng1,21,2020-09-28,Arsenal,Liverpool,1,3,1,2,0,...,1.000000,1.000000,NaN,1.000000,0.500000,NaN,1.000000,0.000000,NaN,0.000000
3,Eng1,21,2020-10-04,Arsenal,Sheff Utd,2,1,0,0,1,...,NaN,0.666667,1.000000,NaN,0.333333,0.000000,NaN,0.000000,0.000000,NaN
4,Eng1,21,2020-10-17,Arsenal,Man City,0,1,0,1,0,...,1.000000,0.750000,NaN,0.500000,0.250000,NaN,0.500000,0.000000,NaN,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16727,Spa1,25,2025-01-20,Villarreal,Mallorca,4,0,4,0,1,...,NaN,0.578947,0.555556,NaN,0.210526,0.333333,NaN,0.105263,0.111111,NaN
16728,Spa1,25,2025-01-25,Villarreal,Atletico,1,1,1,0,0,...,0.700000,0.600000,NaN,0.600000,0.250000,NaN,0.100000,0.150000,NaN,0.100000
16729,Spa1,25,2025-02-01,Villarreal,Valladolid,0,0,0,0,1,...,NaN,0.571429,0.600000,NaN,0.238095,0.400000,NaN,0.142857,0.200000,NaN
16730,Spa1,25,2025-02-08,Villarreal,Las Palmas,0,0,0,0,0,...,0.727273,0.545455,NaN,0.545455,0.227273,NaN,0.090909,0.136364,NaN,0.090909


In [351]:
team_filter = team_df[(team_df["Team"]=="Chelsea")]
team_filter

,country,season,date,Team,Opponent,GoalsScored,GoalsConceded,FirstHalfGoalsScored,FirstHalfGoalsConceded,is_home,...,Away_TeamPct_Over_0.5,TeamPct_Over_1.5,Home_TeamPct_Over_1.5,Away_TeamPct_Over_1.5,TeamPct_Over_2.5,Home_TeamPct_Over_2.5,Away_TeamPct_Over_2.5,TeamPct_Over_3.5,Home_TeamPct_Over_3.5,Away_TeamPct_Over_3.5
190,Eng1,21,2020-09-14,Chelsea,Brighton,3,1,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
191,Eng1,21,2020-09-20,Chelsea,Liverpool,0,2,0,0,1,...,NaN,1.000000,NaN,NaN,1.000000,NaN,NaN,0.000000,NaN,NaN
192,Eng1,21,2020-09-26,Chelsea,West Bromwich,3,3,0,3,0,...,1.000000,0.500000,NaN,1.000000,0.500000,NaN,1.000000,0.000000,NaN,0.000000
193,Eng1,21,2020-10-03,Chelsea,C Palace,4,0,0,0,1,...,NaN,0.666667,0.000000,NaN,0.666667,0.000000,NaN,0.000000,0.000000,NaN
194,Eng1,21,2020-10-17,Chelsea,Southampton,3,3,2,1,1,...,NaN,0.750000,0.500000,NaN,0.750000,0.500000,NaN,0.250000,0.500000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3210,Eng1,25,2025-01-14,Chelsea,Bournemouth,2,2,1,0,1,...,NaN,0.450000,0.444444,NaN,0.300000,0.222222,NaN,0.200000,0.111111,NaN
3211,Eng1,25,2025-01-20,Chelsea,Wolves,3,1,1,1,1,...,NaN,0.476190,0.500000,NaN,0.285714,0.200000,NaN,0.190476,0.100000,NaN
3212,Eng1,25,2025-01-25,Chelsea,Man City,1,3,1,1,0,...,0.818182,0.500000,NaN,0.454545,0.318182,NaN,0.363636,0.181818,NaN,0.272727
3213,Eng1,25,2025-02-03,Chelsea,West Ham,0,0,0,0,1,...,NaN,0.478261,0.545455,NaN,0.304348,0.272727,NaN,0.173913,0.090909,NaN


In [352]:
# # -----------------------------
# # 1. Process Home-Team Features
# # -----------------------------
# home_subset = team_df[team_df['is_home'] == 1].copy()
# home_subset = home_subset.drop(columns=['Opponent'])
# home_subset.rename(columns={'Team': 'home_team'}, inplace=True)
#
# # Key columns that remain unchanged
# home_key_cols = ['country', 'season', 'date', 'home_team', 'is_home']
#
# # Keep only columns that start with "Overall_" or "Home_"
# home_feature_cols = [col for col in home_subset.columns
#                      if col not in home_key_cols and (col.startswith("Overall_") or col.startswith("Home_"))]
#
# # Create a DataFrame with only key columns and desired features
# home_features = home_subset[home_key_cols + home_feature_cols].copy()
#
# # Rename the feature columns by adding a "home_" prefix
# home_features.rename(columns={col: "home_" + col for col in home_feature_cols}, inplace=True)
#
#
# # -----------------------------
# # 2. Process Away-Team Features
# # -----------------------------
# away_subset = team_df[team_df['is_home'] == 0].copy()
# away_subset = away_subset.drop(columns=['Opponent'])
# away_subset.rename(columns={'Team': 'away_team'}, inplace=True)
#
# # Key columns that remain unchanged
# away_key_cols = ['country', 'season', 'date', 'away_team', 'is_home']
#
# # Keep only columns that start with "Overall_" or "Away_"
# away_feature_cols = [col for col in away_subset.columns
#                      if col not in away_key_cols and (col.startswith("Overall_") or col.startswith("Away_"))]
#
# # Create a DataFrame with only key columns and desired features
# away_features = away_subset[away_key_cols + away_feature_cols].copy()
#
# # Rename the feature columns by adding an "away_" prefix
# away_features.rename(columns={col: "away_" + col for col in away_feature_cols}, inplace=True)
#
#
# # -----------------------------
# # 3. Merge Back into the Match-Level DataFrame
# # -----------------------------
# match_df = data.copy()
#
# # Merge home features on common keys
# match_df = match_df.merge(home_features, on=['country', 'season', 'date', 'home_team'], how='left')
#
# # Merge away features on common keys
# match_df = match_df.merge(away_features, on=['country', 'season', 'date', 'away_team'], how='left')
#
# match_df


,country,season,date,ko_time,round,home_team,away_team,home_goals_ft,away_goals_ft,home_goals_ht,...,away_Away_Percent_Over_2.5,away_Away_Rolling5_Percent_Over_2.5,away_Overall_Percent_Over_3.5,away_Overall_Rolling5_Percent_Over_3.5,away_Away_Percent_Over_3.5,away_Away_Rolling5_Percent_Over_3.5,away_Away_TeamPct_Over_0.5,away_Away_TeamPct_Over_1.5,away_Away_TeamPct_Over_2.5,away_Away_TeamPct_Over_3.5
0,Fra1,21,2020-08-21,1800,1,Bordeaux,Nantes,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Fra1,21,2020-08-22,2000,1,Lille,Rennes,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Fra1,21,2020-08-22,1600,1,Dijon,Angers,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fra1,21,2020-08-23,1200,1,AS Monaco,Reims,2,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Fra1,21,2020-08-23,1400,1,Lorient,Strasbourg,3,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8361,Ger1,25,2025-02-16,1430,22,Bremen,Hoffenheim,0,0,0,...,0.1,0.2,0.047619,0.0,0.0,0.0,0.500000,0.100000,0.1,0.0
8362,Ger1,25,2025-02-16,1830,22,Heidenheim,1. FSV Mainz 05,0,0,0,...,0.6,0.4,0.000000,0.0,0.0,0.0,0.700000,0.600000,0.6,0.0
8363,Ger1,25,2025-02-16,1630,22,Frankfurt,Kiel,0,0,0,...,0.0,0.0,0.095238,0.2,0.0,0.0,0.900000,0.500000,0.0,0.0
8364,Ita1,25,2025-02-16,1400,25,Parma Calcio,Roma,0,0,0,...,0.0,0.0,0.083333,0.0,0.0,0.0,0.583333,0.250000,0.0,0.0


In [353]:
team_filter2 = match_df[(match_df["away_team"]=="West Bromwich")]
team_filter2

,country,season,date,ko_time,round,home_team,away_team,home_goals_ft,away_goals_ft,home_goals_ht,...,away_Away_Percent_Over_2.5,away_Away_Rolling5_Percent_Over_2.5,away_Overall_Percent_Over_3.5,away_Overall_Rolling5_Percent_Over_3.5,away_Away_Percent_Over_3.5,away_Away_Rolling5_Percent_Over_3.5,away_Away_TeamPct_Over_0.5,away_Away_TeamPct_Over_1.5,away_Away_TeamPct_Over_2.5,away_Away_TeamPct_Over_3.5
60,Eng1,21,2020-09-19,1230,2,Everton,West Bromwich,5,2,2,...,NaN,NaN,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN
183,Eng1,21,2020-10-04,1200,4,Southampton,West Bromwich,2,0,1,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,1.000000,1.000000,0.000000,0.000000
289,Eng1,21,2020-10-26,1730,6,Brighton,West Bromwich,1,1,1,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.500000,0.500000,0.000000,0.000000
339,Eng1,21,2020-11-02,1730,7,Fulham,West Bromwich,2,0,2,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.666667,0.333333,0.000000,0.000000
409,Eng1,21,2020-11-21,2000,9,Man Utd,West Bromwich,1,0,0,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.500000,0.250000,0.000000,0.000000
556,Eng1,21,2020-12-12,1500,12,Newcastle,West Bromwich,2,1,1,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.400000,0.200000,0.000000,0.000000
591,Eng1,21,2020-12-15,2000,13,Man City,West Bromwich,1,1,1,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.500000,0.166667,0.000000,0.000000
713,Eng1,21,2020-12-27,1630,15,Liverpool,West Bromwich,1,1,1,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.571429,0.142857,0.000000,0.000000
852,Eng1,21,2021-01-16,1230,19,Wolves,West Bromwich,2,3,2,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.625000,0.125000,0.000000,0.000000
882,Eng1,21,2021-01-19,1800,18,West Ham,West Bromwich,2,1,1,...,0.111111,0.2,0.000000,0.0,0.000000,0.0,0.666667,0.222222,0.111111,0.000000


In [354]:
match_df.dropna(inplace=True)

In [355]:
match_df['total_goals'] = match_df['home_goals_ft'] + match_df['away_goals_ft']
match_df['target'] = match_df['total_goals'].apply(lambda x: 1 if x > 2.5 else 0)
match_df

,country,season,date,ko_time,round,home_team,away_team,home_goals_ft,away_goals_ft,home_goals_ht,...,away_Overall_Percent_Over_3.5,away_Overall_Rolling5_Percent_Over_3.5,away_Away_Percent_Over_3.5,away_Away_Rolling5_Percent_Over_3.5,away_Away_TeamPct_Over_0.5,away_Away_TeamPct_Over_1.5,away_Away_TeamPct_Over_2.5,away_Away_TeamPct_Over_3.5,total_goals,target
88,Fra1,21,2020-09-25,2000,5,Lille,Nantes,2,0,1,...,0.000000,0.00,0.000000,0.0,0.500000,0.000000,0.000000,0.000000,2,0
95,Fra1,21,2020-09-26,1600,5,Saint-Etienne,Rennes,0,3,0,...,0.250000,0.25,0.500000,0.5,1.000000,0.500000,0.500000,0.500000,3,1
96,Fra1,21,2020-09-26,2000,5,Ol. Marseille,Metz,1,1,0,...,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,2,0
111,Fra1,21,2020-09-27,1400,5,Nimes,Lens,1,1,0,...,0.000000,0.00,0.000000,0.0,1.000000,0.500000,0.500000,0.000000,2,0
112,Fra1,21,2020-09-27,1400,5,AS Monaco,Strasbourg,3,2,2,...,0.000000,0.00,0.000000,0.0,0.500000,0.000000,0.000000,0.000000,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8275,Ita1,25,2025-02-02,1130,23,Juventus,Empoli,0,0,0,...,0.045455,0.00,0.090909,0.2,0.909091,0.363636,0.090909,0.090909,0,0
8276,Spa1,25,2025-02-03,2000,22,Girona,Las Palmas,0,0,0,...,0.000000,0.00,0.000000,0.0,0.700000,0.300000,0.200000,0.000000,0,0
8277,Ita1,25,2025-02-03,1945,23,Cagliari,Lazio,0,0,0,...,0.045455,0.00,0.090909,0.0,0.818182,0.363636,0.272727,0.090909,0,0
8278,Eng1,25,2025-02-03,2000,24,Chelsea,West Ham,0,0,0,...,0.043478,0.00,0.000000,0.0,0.909091,0.181818,0.000000,0.000000,0,0


In [356]:
features = [
    # General
    'round',
    'home_team_place_total',
    'home_team_place_home',
    'away_team_place_total',
    'away_team_place_away',
    'home_odds',
    'draw_odds',
    'away_odds',
    'over_25_odds',
    'under_25_odds',
    'elo_home',
    'elo_away',
    'form_home',
    'form_away',

    # Home team overall features
    'home_Overall_RollingGoalsScored_Mean',
    'home_Overall_RollingGoalsConceded_Mean',
    'home_Overall_RollingGoalsScored_Std',
    'home_Overall_RollingGoalsConceded_Std',
    'home_Overall_RollingGoalsScored_Mean_Short',
    'home_Overall_Momentum_GoalsScored',
    'home_Overall_Trend_Slope_GoalsScored',
    'home_Overall_RollingFirstHalfGoalsScored_Mean',
    'home_Overall_RollingFirstHalfGoalsConceded_Mean',
    'home_Overall_RollingFirstHalfGoalsScored_Std',
    'home_Overall_RollingFirstHalfGoalsConceded_Std',
    'home_Overall_RollingFirstHalfGoalsScored_Mean_Short',
    'home_Overall_Momentum_FirstHalfGoalsScored',
    'home_Overall_Trend_Slope_FirstHalfGoalsScored',

    # Home team specific features (home-only)
    'home_Home_RollingGoalsScored_Mean',
    'home_Home_RollingGoalsConceded_Mean',
    'home_Home_RollingGoalsScored_Std',
    'home_Home_RollingGoalsConceded_Std',
    'home_Home_RollingGoalsScored_Mean_Short',
    'home_Home_Momentum_GoalsScored',
    'home_Home_Trend_Slope_GoalsScored',
    'home_Home_RollingFirstHalfGoalsScored_Mean',
    'home_Home_RollingFirstHalfGoalsConceded_Mean',
    'home_Home_RollingFirstHalfGoalsScored_Std',
    'home_Home_RollingFirstHalfGoalsConceded_Std',
    'home_Home_RollingFirstHalfGoalsScored_Mean_Short',
    'home_Home_Momentum_FirstHalfGoalsScored',
    'home_Home_Trend_Slope_FirstHalfGoalsScored',

    # Away team overall features
    'away_Overall_RollingGoalsScored_Mean',
    'away_Overall_RollingGoalsConceded_Mean',
    'away_Overall_RollingGoalsScored_Std',
    'away_Overall_RollingGoalsConceded_Std',
    'away_Overall_RollingGoalsScored_Mean_Short',
    'away_Overall_Momentum_GoalsScored',
    'away_Overall_Trend_Slope_GoalsScored',
    'away_Overall_RollingFirstHalfGoalsScored_Mean',
    'away_Overall_RollingFirstHalfGoalsConceded_Mean',
    'away_Overall_RollingFirstHalfGoalsScored_Std',
    'away_Overall_RollingFirstHalfGoalsConceded_Std',
    'away_Overall_RollingFirstHalfGoalsScored_Mean_Short',
    'away_Overall_Momentum_FirstHalfGoalsScored',
    'away_Overall_Trend_Slope_FirstHalfGoalsScored',

    # Away team specific features (away-only)
    'away_Away_RollingGoalsScored_Mean',
    'away_Away_RollingGoalsConceded_Mean',
    'away_Away_RollingGoalsScored_Std',
    'away_Away_RollingGoalsConceded_Std',
    'away_Away_RollingGoalsScored_Mean_Short',
    'away_Away_Momentum_GoalsScored',
    'away_Away_Trend_Slope_GoalsScored',
    'away_Away_RollingFirstHalfGoalsScored_Mean',
    'away_Away_RollingFirstHalfGoalsConceded_Mean',
    'away_Away_RollingFirstHalfGoalsScored_Std',
    'away_Away_RollingFirstHalfGoalsConceded_Std',
    'away_Away_RollingFirstHalfGoalsScored_Mean_Short',
    'away_Away_Momentum_FirstHalfGoalsScored',
    'away_Away_Trend_Slope_FirstHalfGoalsScored',

    # ----- Additional Goal Threshold Percentages (Per-Match Metrics) -----
    # For thresholds: 1.5, 2.5, 3.5
    # Overall team (season cumulative and rolling last 5 matches)
    'home_Overall_Percent_Over_1.5',
    'home_Overall_Rolling5_Percent_Over_1.5',
    'home_Overall_Percent_Over_2.5',
    'home_Overall_Rolling5_Percent_Over_2.5',
    'home_Overall_Percent_Over_3.5',
    'home_Overall_Rolling5_Percent_Over_3.5',

    'away_Overall_Percent_Over_1.5',
    'away_Overall_Rolling5_Percent_Over_1.5',
    'away_Overall_Percent_Over_2.5',
    'away_Overall_Rolling5_Percent_Over_2.5',
    'away_Overall_Percent_Over_3.5',
    'away_Overall_Rolling5_Percent_Over_3.5',

    # Home matches only
    'home_Home_Percent_Over_1.5',
    'home_Home_Rolling5_Percent_Over_1.5',
    'home_Home_Percent_Over_2.5',
    'home_Home_Rolling5_Percent_Over_2.5',
    'home_Home_Percent_Over_3.5',
    'home_Home_Rolling5_Percent_Over_3.5',

    # Away matches only
    'away_Away_Percent_Over_1.5',
    'away_Away_Rolling5_Percent_Over_1.5',
    'away_Away_Percent_Over_2.5',
    'away_Away_Rolling5_Percent_Over_2.5',
    'away_Away_Percent_Over_3.5',
    'away_Away_Rolling5_Percent_Over_3.5',


    # Home matches only
    'home_Home_TeamPct_Over_0.5',
    'home_Home_TeamPct_Over_1.5',
    'home_Home_TeamPct_Over_2.5',
    'home_Home_TeamPct_Over_3.5',

    # Away matches only
    'away_Away_TeamPct_Over_0.5',
    'away_Away_TeamPct_Over_1.5',
    'away_Away_TeamPct_Over_2.5',
    'away_Away_TeamPct_Over_3.5'
]
print(f"Number of features: {len(features)}")

In [359]:
# Process each league separately
leagues = match_df[['country']].drop_duplicates().apply(tuple, axis=1)

for league in leagues:
    print(f"Processing league: {league}")
    # Filter matches for the current league
    matches_filtered = match_df[match_df['country'] == league[0]]

    # Use the filtered data for features and target
    X = matches_filtered[features]
    y = matches_filtered['target']  # ensure this is a categorical variable for classification

    # Time-series split: first 80% for training, last 20% for testing
    train_size = int(len(matches_filtered) * 0.8)
    X_train = X.iloc[:train_size]
    X_test = X.iloc[train_size:]
    y_train = y.iloc[:train_size]
    y_test = y.iloc[train_size:]

    # Train a RandomForestClassifier
    model = RandomForestClassifier(random_state=42)
    model.fit(X_train, y_train)

    # Evaluate permutation importance using precision as the scoring metric
    result = permutation_importance(model, X_test, y_test, n_repeats=10, random_state=42, scoring='precision')

    # Display feature importances
    print("Feature importances:")
    for i in result.importances_mean.argsort()[::-1]:
        print(f"{X.columns[i]}: {result.importances_mean[i]:.3f} +/- {result.importances_std[i]:.3f}")


Processing league: ('Fra1',)
Feature importances:
home_Home_TeamPct_Over_2.5: 0.016 +/- 0.005
home_Home_Momentum_FirstHalfGoalsScored: 0.013 +/- 0.008
away_Overall_Rolling5_Percent_Over_1.5: 0.011 +/- 0.007
away_Away_Momentum_GoalsScored: 0.009 +/- 0.007
home_Home_RollingFirstHalfGoalsScored_Mean_Short: 0.009 +/- 0.005
away_Overall_Momentum_GoalsScored: 0.008 +/- 0.008
elo_home: 0.008 +/- 0.009
away_Away_Percent_Over_3.5: 0.008 +/- 0.006
home_Home_RollingFirstHalfGoalsConceded_Mean: 0.008 +/- 0.008
home_Home_Momentum_GoalsScored: 0.008 +/- 0.005
home_Overall_RollingFirstHalfGoalsConceded_Mean: 0.007 +/- 0.007
under_25_odds: 0.007 +/- 0.014
away_Overall_RollingFirstHalfGoalsConceded_Mean: 0.006 +/- 0.008
home_Overall_RollingFirstHalfGoalsConceded_Std: 0.006 +/- 0.006
home_Home_RollingGoalsScored_Std: 0.006 +/- 0.007
home_Home_RollingGoalsConceded_Std: 0.006 +/- 0.007
over_25_odds: 0.006 +/- 0.011
away_Away_TeamPct_Over_1.5: 0.006 +/- 0.008
home_Home_Rolling5_Percent_Over_1.5: 0.006 +/- 

In [360]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier

# Process each league separately
leagues = match_df[['country']].drop_duplicates().apply(tuple, axis=1)

for league in leagues:
    print(f"Processing league: {league[0]}")
    # Filter the matches for the current league
    matches_filtered = match_df[match_df['country'] == league[0]]

    # Extract features and target for the current league
    X = matches_filtered[features]
    y = matches_filtered['target']

    # Initialize the RandomForestClassifier as the estimator
    estimator = RandomForestClassifier(random_state=42)

    # Create the RFE selector to keep the top 20 features
    selector = RFE(estimator, n_features_to_select=20, step=1)
    selector = selector.fit(X, y)

    # Get the names of the selected features
    selected_features = X.columns[selector.support_]
    print(f"Top 20 features for league {league[0]}:")
    print(selected_features.tolist(), "\n")


Processing league: Fra1
Top 20 features for league Fra1:
['round', 'home_odds', 'draw_odds', 'away_odds', 'over_25_odds', 'under_25_odds', 'elo_home', 'elo_away', 'form_home', 'form_away', 'home_Overall_Momentum_GoalsScored', 'home_Home_RollingGoalsScored_Std', 'away_Overall_RollingGoalsConceded_Std', 'away_Overall_Momentum_GoalsScored', 'away_Away_RollingGoalsConceded_Std', 'away_Away_Momentum_GoalsScored', 'home_Overall_Percent_Over_1.5', 'home_Overall_Percent_Over_2.5', 'away_Overall_Percent_Over_1.5', 'away_Overall_Percent_Over_2.5'] 

Processing league: Spa1
Top 20 features for league Spa1:
['home_odds', 'draw_odds', 'away_odds', 'over_25_odds', 'under_25_odds', 'elo_home', 'elo_away', 'form_home', 'form_away', 'home_Overall_RollingGoalsConceded_Std', 'home_Overall_Momentum_GoalsScored', 'home_Home_RollingGoalsScored_Std', 'away_Overall_RollingGoalsScored_Std', 'away_Overall_Momentum_GoalsScored', 'away_Away_Momentum_GoalsScored', 'home_Overall_Percent_Over_1.5', 'away_Overall_Per